In [4]:
import pandas as pd
import numpy as np
from torch import nn
import torch
from torch.utils.data import Dataset
from typing import Callable, 
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
df = pd.read_hdf("apr21a.hdf")
df = df.fillna(0)
df["trade_in_1h_usdtfutures_ADAUSDT_close_price"].values
df

,trade_in_3h_usdtfutures_ADAUSDT_close_price,trade_in_2h_usdtfutures_ADAUSDT_close_price,trade_in_1h_usdtfutures_ADAUSDT_close_price,usdtfutures_ADAUSDT_close_price,usdtfutures_ADAUSDT_volume,usdtfutures_ADAUSDT_number_of_trades,usdtfutures_ADAUSDT_close_price_ewm_mean,usdtfutures_ADAUSDT_close_price_ewm_std,usdtfutures_ADAUSDT_close_price_ewm_cov,usdtfutures_ADAUSDT_close_price_mean_2,...,usdtfutures_ADAUSDT_number_of_trades_std_128,usdtfutures_ADAUSDT_number_of_trades_cov_128,usdtfutures_ADAUSDT_number_of_trades_mean_256,usdtfutures_ADAUSDT_number_of_trades_skew_256,usdtfutures_ADAUSDT_number_of_trades_kurt_256,usdtfutures_ADAUSDT_number_of_trades_std_256,usdtfutures_ADAUSDT_number_of_trades_cov_256,aa1,aa2,aa3
2020-08-05 05:00:59.999000+00:00,-0.002483,-0.000966,-0.000207,0.14497,221765.0,135,0.144996,0.000109,1.196156e-08,0.145045,...,87.342755,7628.756828,102.734375,2.054330,4.349153,82.949593,6880.635049,0.001035,-0.000828,-0.000207
2020-08-05 05:01:59.999000+00:00,-0.002208,-0.002277,-0.000759,0.14494,129869.0,71,0.144959,0.000073,5.398875e-09,0.144955,...,87.338635,7628.037156,102.621094,2.056488,4.352256,82.973141,6884.542142,0.000207,0.001242,-0.000621
2020-08-05 05:02:59.999000+00:00,-0.002002,-0.001450,-0.001519,0.14483,206899.0,91,0.144873,0.000096,9.171132e-09,0.144885,...,87.174906,7599.464321,102.609375,2.056802,4.352926,82.974575,6884.780147,0.000760,0.000967,0.002002
2020-08-05 05:03:59.999000+00:00,0.000761,-0.000484,0.000069,0.14461,172555.0,146,0.144698,0.000184,3.378216e-08,0.144720,...,87.165062,7597.748031,102.703125,2.051261,4.331146,83.010121,6890.680147,0.001521,0.002282,0.002489
2020-08-05 05:04:59.999000+00:00,0.000899,0.000691,-0.000553,0.14462,194681.0,150,0.144646,0.000118,1.394002e-08,0.144615,...,85.954978,7388.258305,103.128906,2.040936,4.301199,82.971924,6884.340181,-0.000069,0.001452,0.002213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-24 23:56:59.999000+00:00,-0.000363,-0.000605,-0.000484,0.08264,46377.0,79,0.082640,0.000006,3.578294e-11,0.082640,...,85.862243,7372.324742,85.000000,4.001880,22.237342,68.350653,4671.811765,0.000000,0.000000,0.000242
2020-09-24 23:57:59.999000+00:00,-0.000121,0.000121,-0.000121,0.08260,28640.0,80,0.082613,0.000027,7.384971e-10,0.082620,...,85.754405,7353.817913,85.128906,4.004035,22.266729,68.309812,4666.230377,0.000484,0.000484,0.000484
2020-09-24 23:58:59.999000+00:00,0.000000,0.000000,0.000242,0.08259,29488.0,23,0.082598,0.000022,4.911399e-10,0.082595,...,85.977751,7392.173659,84.960938,3.991049,22.161573,68.409826,4679.904350,0.000121,0.000605,0.000605
2020-09-24 23:59:59.999000+00:00,0.000000,0.000000,-0.000242,0.08261,291284.0,106,0.082606,0.000015,2.295846e-10,0.082600,...,85.900106,7378.828187,85.242188,3.991294,22.185611,68.347468,4671.376409,-0.000242,-0.000121,0.000363


In [6]:
class InputDataset(Dataset):
    def __init__(
        self, df: pd.DataFrame, device: torch.DeviceObjType, transform: Callable = None
    ):
        self.seq_len=64
        self.df = df
        self.len = len(self.df) - self.seq_len
        self.transform = transform
        self.device = device
    def __getitem__(self, index: int) -> Tuple[Tensor, Tensor]:
        y_index = index + self.seq_len
        x = torch.from_numpy(df["trade_in_1h_usdtfutures_ADAUSDT_close_price"].values)[index:y_index-1]
        y = torch.from_numpy(df["trade_in_1h_usdtfutures_ADAUSDT_close_price"].values)[y_index]
        x = x.to(self.device)
        y = y.to(self.device)
        return x, y
        
    def __len__(self):
        return self.len

validation_size = int(len(df)/5)
dataset_train = InputDataset(df=df.iloc[:-validation_size], device=DEVICE)
dataset_valid = InputDataset(df=df.iloc[-validation_size:], device=DEVICE)





NameError: name 'Callable' is not defined

In [3]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        input_seq = input_seq.view(len(input_seq) ,1, -1)
        lstm_out, self.hidden_cell = self.lstm(input_seq, self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [4]:
model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
input_seq = df["trade_in_1h_usdtfutures_ADAUSDT_close_price"].values
seq_len = 10
split_len = int(len(input_seq) / 10)-1
input_seq = np.array_split(input_seq, split_len)
input_seq = input_seq[:-1]
input_seq = input_seq[1:]


In [11]:
input_seq =torch.Tensor(input_seq)

train_inout_seq = []
for i in range(len(input_seq)):
    if i==0:
        continue
    tis = [input_seq[i-1], [input_seq[i][0]]]
    train_inout_seq.append(tis)
train_inout_seq

ValueError: expected sequence of length 11 at dim 1 (got 10)

In [ ]:
epochs = 150




for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')